In [ ]:
import pandas as pd
import numpy as np
import scipy as sp
import plotly
import plotly.offline as po
import plotly.express as px
from scipy import stats
import plotly.graph_objects as go

In [ ]:
csv_list = ['1-1.csv', '1-2.csv', '1-3.csv', '2-1.csv', '2-2.csv', '2-3.csv',
            '4-1.csv', '4-2.csv', '4-3.csv', '5-1.csv', '5-2.csv', '5-3.csv',
            '6-1.csv', '6-2.csv', '6-3.csv', '7-1.csv', '7-2.csv', '7-3.csv',
            '9-1.csv', '9-2.csv', '9-3.csv', '10-1.csv', '10-2.csv']

In [ ]:
axis_list = []
axis_list_2 = []
axis_name = []
for i in range(len(csv_list)):
    axis_list.append('y'+str(i + 1))
    axis_list_2.append('y'+str(i + 1 + len(csv_list)))
    axis_name.append('yaxis'+str(i + 1))

In [ ]:
plot_list = []

fig = plotly.subplots.make_subplots(rows=len(csv_list), shared_xaxes=False, subplot_titles=csv_list)

for file in csv_list:
    
    filename = file
    
    print(filename)
    
    df = pd.read_csv(filename, delimiter=',', low_memory=False)
    df = pd.DataFrame(df)
    df.columns = df.iloc[9]
    df['Run Time'] = pd.Series(range(1, df.shape[0] + 1, 1))
    df = df[10:]
    df = df.astype(float)
    df['dRdt'] = (10**3)*(df['Resistance End (Ω)'] - df['Resistance Start (Ω)'])/(df['Pulse End (ms)'] - df['Pulse Start (ms)'])
    df['Puff Count'] = df['Puff Count'].astype(int)
    
    df
    
#     for i in range(1, df['Puff Count'].max() + 1, 1):
#         vals = df[df["Puff Count"].isin([i])]
#         df.loc[df["Puff Count"].isin([i]), 'Max dRdt'] = vals['dRdt'].max().round(decimals=0)
    
    for i in range(1, df['Puff Count'].max() + 1, 1):
        vals = df[df["Puff Count"].isin([i])]
        local_max = vals['dRdt'].max().round(decimals=0)
        
        if i == 1:
            run_avg = local_max
        else:
            run_avg = ((i - 1)*run_avg + local_max)/i
        
        if local_max/run_avg >= 1.25 and local_max >= run_avg + 1.5:
            dry_wick = 500
        else:
            dry_wick = 0
        
        df.loc[df["Puff Count"].isin([i]), 'Max dRdt'] = local_max
        df.loc[df["Puff Count"].isin([i]), 'Dry Wick'] = dry_wick
        
    
    df['Test'] = file
    
    
    # ///////////////////////////////////// PLOTTING ////////////////////////////////////////////
    
    
    plot_list.append(go.Scatter3d(x=df['Test'][::10], y=df['Run Time'][::10], z=df['dRdt'][::10], mode='markers',
                             marker=dict(
                                        size=2,
                                        color=df['Dry Wick'][::10], #set color equal to a variable
                                        colorscale='Viridis', # one of plotly colorscales
                                        showscale=False
                                    )))
    
    plot_list.append(go.Scatter3d(x=df['Test'][::10], y=df['Run Time'][::10], z=df['Max dRdt'][::10], mode='lines'))
    
    fig.add_trace(go.Scattergl(x=df['Run Time'], 
                         y=df['dRdt'], 
                         name=file[:-4],
                         mode='markers',
                         marker=dict(
                                    size=2,
                                    color=df['Dry Wick'], #set color equal to a variable
                                    colorscale='Viridis', # one of plotly colorscales
                                    showscale=False
                                    )),
              row=csv_list.index(file) + 1,
              col=1)
    
    fig.add_trace(go.Scattergl(x=df['Run Time'], 
                     y=df['Max dRdt'], 
                     name='Max ' + file[:-4],
                     mode='lines',
                     ),
          row=csv_list.index(file) + 1,
          col=1)
    
    temp_ax = axis_list_2[csv_list.index(file)]
    
    fig.add_trace(go.Scattergl(x=df['Run Time'], y=df['Temperature (C)']/10, mode='lines',
                           name=file + ' Temp',
                           marker=dict(size=2)),
                           row=csv_list.index(file) + 1,
                           col=1)
    
    
    
    fig.update_yaxes(title_text="dRdt (milliOhms/s)", row=csv_list.index(file) + 1, range=[-5, 80])
    
#     globals()[axis_name[csv_list.index(file)]]
    
#     fig.update_layout(
#                             yaxis=dict(
#                             title='Temperature (C)',
#                             anchor='free',
#                             overlaying=axis_list[csv_list.index(file)],
#                             side='right',),
        
#     )

    
fig.update_layout(height=10000, width=1000, title_text="P4 dR/dt Dry Wick Detection")

fig.show()
    
    
# big_fig = go.Figure(data=plot_list)

# big_fig.update_layout(
#     title=str(filename),
#     xaxis_title="(Test No.)",
#     yaxis_title="Run Time (~10ms)",
#     scene = dict(
#         xaxis = dict(nticks=len(csv_list)),
#         zaxis = dict(range=[-20,100],),)
#     )
